## Nick Elias Capstone Project - Grocery Data Analysis

## Introduction
This project focuses on analyzing grocery price data to identify trends and insights related to product pricing over time. The analysis is conducted using SQLite for data management and Pandas for data manipulation and visualization. 

## Data Sources
- **grocery_data.csv**: Contains raw grocery pricing data, including product IDs, pricing, and time series information.
- **product_mapping.csv**: Maps product IDs to product names for better interpretability.

## Process Overview
1. **Data Ingestion**: Import CSV data into Pandas DataFrames and write them to SQLite tables.
2. **Data Cleaning**: 
   - Create a mapping table to convert month codes into month names.
   - Generate a cleaned version of the grocery data by renaming headers and filtering necessary columns.
   - Calculate additional metrics such as average price per year and percentage changes in pricing.
3. **Data Analysis**: 
   - Analyze the highest percentage price changes for each product year-over-year.
   - Create a SQLite view for easy access to the cleaned and processed data.
4. **Visualization**: Use Pandas to visualize and present the findings effectively.

## Summary of Findings
The analysis reveals significant trends in grocery pricing over the observed years, highlighting products with the most considerable fluctuations in price. This information is valuable for stakeholders in understanding pricing dynamics and making informed decisions regarding inventory and pricing strategies.

In [1]:
# Importing sqlite and pandas libraries to run SQL
import sqlite3
import pandas as pd
import os

# Assuming the notebook is in the same directory as the 'data' folder
notebook_directory = os.getcwd()
csv_path = os.path.join(notebook_directory, 'data', 'grocery_data.csv')

# Connect to SQLite database (create if not exists)
conn = sqlite3.connect('GroceryAnalysis.db')
cursor = conn.cursor()

# Read CSV files into pandas DataFrames using the csv_path variable
grocery_data = pd.read_csv(csv_path)
product_mapping = pd.read_csv('data/product_mapping.csv')

# Write DataFrames to SQLite tables
grocery_data.to_sql('cleaned_grocery_data', conn, index=False, if_exists='replace')
product_mapping.to_sql('product_mapping', conn, index=False, if_exists='replace')

# Commit changes and close connection
conn.commit()



In [2]:
#Create a month mapping table to translate month codes (M01) to names (January)
query1 = """
CREATE TABLE IF NOT EXISTS month_mapping (
    month_code VARCHAR(3) PRIMARY KEY,
    month_name VARCHAR(255)
);
"""
cursor.execute(query1)
conn.commit()

In [3]:
#Populate the month mapping table
query2 = """
INSERT OR IGNORE INTO month_mapping (month_code, month_name) VALUES
    ('M01', 'January'),
    ('M02', 'February'),
    ('M03', 'March'),
    ('M04', 'April'),
    ('M05', 'May'),
    ('M06', 'June'),
    ('M07', 'July'),
    ('M08', 'August'),
    ('M09', 'September'),
    ('M10', 'October'),
    ('M11', 'November'),
    ('M12', 'December');
"""
cursor.execute(query2)
conn.commit()

In [4]:
#Clearing cleaned_grocery_data table for a fresh run 
query3 = """
DROP TABLE IF EXISTS cleaned_grocery_data;
"""
cursor.execute(query3)
conn.commit()

In [5]:
#Creating a new grocery data table with cleaned data
query4 = """
CREATE TABLE IF NOT EXISTS cleaned_grocery_data(
    product_id VARCHAR(255),
    product_name VARCHAR(255),
    year INT,
    month_name VARCHAR(255),
    price_usd FLOAT
);
"""
cursor.execute(query4)
conn.commit()

In [6]:
#Populating cleaned grocery table and renaming headers
query5 = """
INSERT INTO cleaned_grocery_data (product_id, year, month_name, price_usd)
SELECT
    Series_id AS product_id,
    Year AS year,
    Period AS month_name,
    Value AS price_usd
FROM
    grocery_data gd;
"""
cursor.execute(query5)
conn.commit()

In [7]:
#Adding in the product names (Eggs, Bread, etc.) from the separate product mapping table to help identify products
query6 = """
UPDATE cleaned_grocery_data
SET product_name = (
    SELECT product_name
    FROM product_mapping
    WHERE product_mapping.product_id = cleaned_grocery_data.product_id
)
WHERE cleaned_grocery_data.product_id IN (
    SELECT product_id
    FROM product_mapping
);
"""
cursor.execute(query6)
conn.commit()

In [8]:
#Adding a new column that will calculate the average price per year for each product 
query7 = """
ALTER TABLE cleaned_grocery_data 
ADD COLUMN avg_price_per_year FLOAT;
"""
cursor.execute(query7)
conn.commit()

In [9]:
#Populating the new average price column and rounding to hundredths for USD 
query8 = """
UPDATE cleaned_grocery_data
SET avg_price_per_year = ROUND(
    (
        SELECT AVG(price_usd)
        FROM cleaned_grocery_data sub
        WHERE sub.product_id = cleaned_grocery_data.product_id
          AND sub.year = cleaned_grocery_data.year
        GROUP BY sub.year
    ),
    2
);
"""
cursor.execute(query8)
conn.commit()

In [10]:
#Calculating the percent change from year to year for each product based on yearly average:
query9 = """
SELECT
    product_id,
    product_name,
    year,
    MAX(percent_change) AS highest_percentage_change
FROM (
    SELECT
        cg.product_id,
        cg.year,
        cg.price_usd,
        pm.product_name,
        ((avg_price - lag_avg_price) / lag_avg_price) * 100 AS percent_change
    FROM cleaned_grocery_data cg
    JOIN product_mapping pm ON cg.product_id = pm.product_id
    JOIN (
        SELECT
            product_id,
            year,
            AVG(price_usd) AS avg_price,
            lag(AVG(price_usd)) OVER (PARTITION BY product_id ORDER BY year) AS lag_avg_price
        FROM cleaned_grocery_data
        GROUP BY product_id, year
    ) avg_prices ON cg.product_id = avg_prices.product_id AND cg.year = avg_prices.year
) changes
GROUP BY product_id, year;
"""
cursor.execute(query9)
conn.commit()

In [11]:
# Selecting the year for each product that has the highest percentage change based on yearly average
query10 = """
WITH RankedChanges AS (
    SELECT
        product_id,
        product_name,
        year,
        (price_usd - lag_avg_price) / lag_avg_price * 100 AS percentage_change,
        RANK() OVER (PARTITION BY product_id ORDER BY (price_usd - lag_avg_price) / lag_avg_price DESC) AS rank_change
    FROM (
        SELECT
            cg.product_id,
            cg.year,
            cg.price_usd,
            pm.product_name,
            LAG(avg_price) OVER (PARTITION BY cg.product_id ORDER BY cg.year) AS lag_avg_price
        FROM cleaned_grocery_data cg
        JOIN product_mapping pm ON cg.product_id = pm.product_id
        JOIN (
            SELECT
                product_id,
                year,
                AVG(price_usd) AS avg_price
            FROM cleaned_grocery_data
            GROUP BY product_id, year
        ) avg_prices ON cg.product_id = avg_prices.product_id AND cg.year = avg_prices.year
    ) changes
)
SELECT
    product_id,
    product_name,
    year
FROM RankedChanges
WHERE rank_change = 1;
"""
cursor.execute(query10)
conn.commit()

In [12]:
#Making a new SQLite view with only desired data, first erasing any previous view with same name
query11 = """
DROP VIEW IF EXISTS grocery_data_view;
"""
cursor.execute(query11)
conn.commit()

In [13]:
#Creating the view with month name, year, product name, product id, rounded USD price, and average price per year
query12 = """
CREATE VIEW grocery_data_view AS
SELECT mm.month_name, cgd.year, cgd.product_name, cgd.product_id, ROUND(cgd.price_usd, 2) AS price_usd, avg_price_per_year
FROM cleaned_grocery_data AS cgd
LEFT JOIN month_mapping AS mm ON cgd.month_name = mm.month_code;
"""
cursor.execute(query12)
conn.commit()

In [14]:
#Displaying the view
query13 = """

SELECT * FROM grocery_data_view;
"""
cursor.execute(query13)
conn.commit()

In [15]:
#Displaying the view with Pandas
query14 = """
SELECT * FROM grocery_data_view;
"""
df = pd.read_sql_query(query14, conn)
df.head(13)

,month_name,year,product_name,product_id,price_usd,avg_price_per_year
0,January,1980,Bacon,APU0000704111,1.45,1.46
1,February,1980,Bacon,APU0000704111,1.41,1.46
2,March,1980,Bacon,APU0000704111,1.36,1.46
3,April,1980,Bacon,APU0000704111,1.32,1.46
4,May,1980,Bacon,APU0000704111,1.27,1.46
5,June,1980,Bacon,APU0000704111,1.27,1.46
6,July,1980,Bacon,APU0000704111,1.37,1.46
7,August,1980,Bacon,APU0000704111,1.47,1.46
8,September,1980,Bacon,APU0000704111,1.62,1.46
9,October,1980,Bacon,APU0000704111,1.64,1.46


In [16]:
#Displaying the product, year with highest percentage change, and percentage change value of that respective year
query15 = """
-- Select the year for each product that has the highest percentage change based on yearly average:
WITH RankedChanges AS (
    SELECT
        product_id,
        product_name,
        year,
        (price_usd - lag_avg_price) / lag_avg_price * 100 AS percentage_change,
        RANK() OVER (PARTITION BY product_id ORDER BY (price_usd - lag_avg_price) / lag_avg_price DESC) AS rank_change
    FROM (
        SELECT
            cg.product_id,
            cg.year,
            cg.price_usd,
            pm.product_name,
            LAG(avg_price) OVER (PARTITION BY cg.product_id ORDER BY cg.year) AS lag_avg_price
        FROM cleaned_grocery_data cg
        JOIN product_mapping pm ON cg.product_id = pm.product_id
        JOIN (
            SELECT
                product_id,
                year,
                AVG(price_usd) AS avg_price
            FROM cleaned_grocery_data
            GROUP BY product_id, year
        ) avg_prices ON cg.product_id = avg_prices.product_id AND cg.year = avg_prices.year
    ) changes
)
SELECT
    product_id,
    product_name,
    year,
    percentage_change
FROM RankedChanges
WHERE rank_change = 1;
"""
df = pd.read_sql_query(query15, conn)
df.head(13)

,product_id,product_name,year,percentage_change
0,APU0000701111,Flour,2008,16.863289
1,APU0000701312,Rice,2008,16.846775
2,APU0000702111,Bread,2023,11.042494
3,APU0000704111,Bacon,1980,16.811743
4,APU0000706111,Chicken,1986,14.730539
5,APU0000708111,Eggs,2023,66.833858
6,APU0000709112,Milk,2009,14.988742
7,APU0000711211,Bananas,1991,21.413721
8,APU0000711311,Oranges,2005,39.385419
9,APU0000712311,Tomatoes,1990,118.560518


In [17]:
#Showing the final cleaned grocery data table
query = "SELECT * FROM cleaned_grocery_data"
df = pd.read_sql_query(query, conn)

# Display the DataFrame
df


,product_id,product_name,year,month_name,price_usd,avg_price_per_year
0,APU0000704111,Bacon,1980,M01,1.453,1.46
1,APU0000704111,Bacon,1980,M02,1.410,1.46
2,APU0000704111,Bacon,1980,M03,1.361,1.46
3,APU0000704111,Bacon,1980,M04,1.323,1.46
4,APU0000704111,Bacon,1980,M05,1.266,1.46
...,...,...,...,...,...,...
7579,APU000074714,Unleaded Gasoline,2023,M06,3.712,3.71
7580,APU000074714,Unleaded Gasoline,2023,M07,3.732,3.71
7581,APU000074714,Unleaded Gasoline,2023,M08,3.955,3.71
7582,APU000074714,Unleaded Gasoline,2023,M09,3.988,3.71
